In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib.gridspec as gridspec
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from IPython.display import HTML
rc('animation', html='html5')

In [2]:
class Rotor:
    def __init__(self, 
                 position=np.array([0.0, 0.0, 0.0]), 
                 axis=np.array([0.0, 0.0, -1.0]), 
                 direction=+1, # +1 for CCW, -1 for CW
                 k_thrust=1.0, 
                 k_torque=0.05):
        self.position = position
        self.axis = axis
        self.direction = direction
        self.k_thrust = k_thrust
        self.k_torque = k_torque
    
    def compute_thrust_and_torque(self, command):
        # Thrust vector
        thrust = self.k_thrust * command * self.axis
        
        # Torque vector generated by rotor thrust
        torque_th = np.cross(self.position, thrust)
        
        # Torque vector generated by rotor drag
        torque_d = - self.k_torque * self.direction * command * self.axis
        
        # Total torque
        torque = torque_th + torque_d
        
        return thrust, torque

In [3]:
class Actuators:
    def __init__(self, 
                 geometry='bicopter'):
        if geometry == 'bicopter':
            positions = np.array([[ 0.0, -0.5, 0.0],  # left
                                  [ 0.0,  0.5, 0.0]]) # right
            axes = np.array([[0.0, 0.0, -1.0] 
                            for _ in range(2)])
            directions = [(-1)**(i+1) 
                          for i in range(2)]
            k_thrust = 3.0
            k_torque = 0.0
            
        elif geometry == 'hexacopter H':
            positions = np.array([[ 0.5, -0.5, 0.0], # front left
                                  [ 0.0, -0.5, 0.0], # mid left
                                  [-0.5, -0.5, 0.0], # rear left
                                  [ 0.5,  0.5, 0.0], # front right
                                  [ 0.0,  0.5, 0.0], # mid right
                                  [-0.5,  0.5, 0.0]])# rear right
            axes = np.array([[0.0, 0.0, -1.0]
                            for _ in range(6)])
            directions = [(-1)**(i+1) 
                          for i in range(6)]
            k_thrust = 1.0
            k_torque = 0.05
            
        # List of rotors
        self.rotors = [Rotor(position=position, 
                             axis=axis, 
                             direction=direction, 
                             k_thrust=k_thrust, 
                             k_torque=k_torque)
                      for position, axis, direction in zip(positions, axes, directions)]
        
        # Actuator effectiveness matrix
        self.effectiveness = np.zeros([4, len(self.rotors)])
        for i, rotor in enumerate(self.rotors):
            # Compute forces and torques applied on the system (body frame)
            thrust, torque = rotor.compute_thrust_and_torque(1.0)
            
            # Fill corresponding line in effectiveness matrix
            self.effectiveness[:, i] = np.array([torque[0], torque[1], torque[2], thrust[2]])
        
    def compute_thrust_and_torque(self, commands):
        output = self.effectiveness.dot(commands)
        
        thrust = np.array([0.0, 0.0, output[3]])
        torque = output[:3]
        
        return thrust, torque

In [4]:
class Control_allocation:
    def __init__(self, actuators):
        self.B = np.linalg.pinv(actuators.effectiveness)
        self.torque_setpoint   = np.array([0.0, 0.0, 0.0])
        self.thrust_setpoint   = np.array([0.0, 0.0, 0.0])
        self.actuator_setpoint = np.zeros(self.B.shape[0])
        
    def compute_actuator_setpoint(self, 
                                  thrust=np.array([0.0, 0.0, 0.0]), 
                                  torque=np.array([0.0, 0.0, 0.0])):
        # Keep setpoints for logging
        self.torque_setpoint = torque
        self.thrust_setpoint = thrust
        
        # Compute output
        actuator_setpoint = self.B.dot(np.array([torque[0], 
                                                 torque[1], 
                                                 torque[2], 
                                                 thrust[2]]))
    
        # Clip actuator setpoint
        self.actuator_setpoint = np.clip(actuator_setpoint, 0.0, 1.0)
        
        return self.actuator_setpoint

In [5]:
class Dynamics:
    def __init__(self, 
                 inertia=0.1*np.eye(3), 
                 mass=0.3):
        # Dynamics
        self.inertia = inertia
        self.mass = mass
        
        # State
        self.position = np.array([0.0, 0.0, 0.0])
        self.velocity = np.array([0.0, 0.0, 0.0])
        self.attitude = np.array([0.0, 0.0, 0.0]) # rpy
        self.rate = np.array([0.0, 0.0, 0.0])
    
    def body_to_local_frame(self, vector):
        roll, pitch, yaw = self.attitude
        cr, sr = np.cos(roll), np.sin(roll)
        cp, sp = np.cos(pitch), np.sin(pitch)
        cy, sy = np.cos(yaw), np.sin(yaw)
        
        # Rotation matrix
        rot_yp = np.array([[cp * cy, cp * sy, -sp],
                           [-sy, cy, 0.0],
                           [sp * cy, sp * sy, cp]]);
        rot_r = np.array([[1.0, 0.0, 0.0],
                          [0.0, cr, -sr],
                          [0.0, sr, cr]])
        rotation = np.dot(rot_yp, rot_r)
        
        return rotation.T.dot(vector)
        
    def update(self, 
               thrust=np.array([0.0, 0.0, 0.0]), 
               torque=np.array([0.0, 0.0, 0.0]),
               dt=0.01):        
        # Compute thrust in local frame
        thrust_lf = self.body_to_local_frame(thrust)

        # Compute drag force
        drag = - 0.3 * 0.1 * np.linalg.norm(self.velocity) * self.velocity
        
        # gravity accel
        gravity = - np.array([0.0, 0.0, -9.81])
        
        # Compute body linear acceleration (local frame)
        accel_lf = (thrust_lf + drag) / self.mass + gravity
        
        # Increment position and velocity
        self.position += self.velocity * dt + 0.5 * thrust_lf * dt**2
        self.velocity += accel_lf * dt
        
        # Compute angular acceleration (in body frame)
        ang_accel = np.linalg.inv(self.inertia).dot(torque)
        
        # Increment rate and angles
        self.attitude += self.rate * dt + 0.5 * ang_accel * dt**2  # This only works for small roll and pitch angles
        self.rate += ang_accel * dt        

In [6]:
class PID:
    def __init__(self, kp=1.0, ki=0.1, kd=0.01):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.last_error = 0.0
        self.accumulator = 0.0
        self.output = 0.0
        
    def update(self, error=0.0, dt=0.01):       
        # Proportional term
        proportional = self.kp * error
        
        # Derivative term
        derivative = self.kd * (error - self.last_error) / dt
        self.last_error = error 
        
        # Integral term
        self.accumulator += error
        integral = self.ki * self.accumulator
        
        # Return sum
        self.output = proportional + derivative + integral
        
        return self.output

In [7]:
class Cascade_controller:
    def __init__(self):
        # PIDs
        self.rate_pid     = [PID(kp=1.25, kd=0.02, ki=0.0),
                             PID(kp=1.25, kd=0.02, ki=0.0),
                             PID(kp=1.45, kd=0.03, ki=0.0)]
        self.attitude_pid = [PID(kp=3.0, kd=0.0, ki=0.0) for _ in range(3)]
        self.velocity_pid = [PID(kp=5.0, kd=1.0, ki=0.0),
                             PID(kp=5.0, kd=1.0, ki=0.0),
                             PID(kp=5.0, kd=0.5, ki=0.1)]
        self.position_pid = [PID(kp=2.5, kd=0.7, ki=0.0) for _ in range(3)]
        
        # Setpoints
        self.position_setpoint = np.array([0.0, 0.0, 0.0])
        self.velocity_setpoint = np.array([0.0, 0.0, 0.0])
        self.acceleration_setpoint = np.array([0.0, 0.0, 0.0])
        self.attitude_setpoint = np.array([0.0, 0.0, 0.0])
        self.rate_setpoint     = np.array([0.0, 0.0, 0.0])
        self.torque_setpoint   = np.array([0.0, 0.0, 0.0])
        self.thrust_setpoint   = np.array([0.0, 0.0, 0.0])
        
    def update_rate(self, 
                    rate_setpoint=np.array([0.0, 0.0, 0.0]), 
                    rate_feedback=np.array([0.0, 0.0, 0.0]), 
                    dt=0.01):
        # Rate control
        self.rate_setpoint = rate_setpoint
        self.torque_setpoint = np.array([pid.update(error=error, dt=dt)
                                        for error, pid in zip(rate_setpoint - rate_feedback, 
                                                              self.rate_pid)])
        return self.torque_setpoint
    
    def update_attitude(self, 
                        attitude_setpoint=np.array([0.0, 0.0, 0.0]), 
                        attitude_feedback=np.array([0.0, 0.0, 0.0]),
                        rate_feedback=np.array([0.0, 0.0, 0.0]),
                        dt=0.01):        
        # Attitude control
        self.attitude_setpoint = attitude_setpoint
        self.rate_setpoint = np.array([pid.update(error=error, dt=dt)
                                    for error, pid in zip(attitude_setpoint - attitude_feedback, 
                                                          self.attitude_pid)])
        # Limit rate setpoint
        self.rate_setpoint = np.clip(self.rate_setpoint, - 2 * np.pi, 2 * np.pi)
        
        # Rate control
        self.torque_setpoint = self.update_rate(rate_setpoint=self.rate_setpoint, 
                                               rate_feedback=rate_feedback, 
                                               dt=dt)
        
        return self.torque_setpoint
    
    def update_velocity(self, 
                        velocity_setpoint=np.array([0.0, 0.0, 0.0]), 
                        velocity_feedback=np.array([0.0, 0.0, 0.0]), 
                        attitude_feedback=np.array([0.0, 0.0, 0.0]),
                        rate_feedback=np.array([0.0, 0.0, 0.0]),
                        dt=0.01):        
        # Velocity control
        self.velocity_setpoint = velocity_setpoint
        self.acceleration_setpoint = np.array([pid.update(error=error, dt=dt)
                                                for error, pid in zip(velocity_setpoint - velocity_feedback, 
                                                                      self.velocity_pid)])
        
        # Compensate effect of gravity
        self.acceleration_setpoint += np.array([0.0, 0.0, -9.81])
        
        # Clip acceleration setpoint (we cannot produce downward thrust)
#         if self.acceleration_setpoint[2] > 0.0:
#         self.acceleration_setpoint[2] = np.clip(self.acceleration_setpoint[2], -10.0, 0.0001)
        
        # Convert acceleration setpoint to thrust and attitude setpoints
        # Here we consider that yaw=0 at all time, 
        # but if yaw was not =0 (like when doing coordinated turns by aligning yaw with the velocity vector),
        # we would need to rotate acceleration_setpoint by -yaw around Z axis (i.e. semi-local frame)
        roll = np.clip(- np.arctan2(self.acceleration_setpoint[1], 
                                    np.abs(self.acceleration_setpoint[2])),
                       -0.5, 0.5)
        pitch = np.clip(- np.arctan2(self.acceleration_setpoint[0], 
                                     np.abs(self.acceleration_setpoint[2])),
                        -0.5, 0.5)
        yaw = 0.0
        self.attitude_setpoint = np.array([roll, pitch, yaw])
        
        # Note that thrust setpoint is expressed in body frame, so we keep X and Y components equal to 0
        # Mass must be known to convert acceleration to thrust
        mass = 0.3
        self.thrust_setpoint = np.array([0.0,
                                         0.0,
                                         - np.linalg.norm(self.acceleration_setpoint) * mass])
        
        # Attitude + Rate control
        self.torque_setpoint = self.update_attitude(attitude_setpoint=self.attitude_setpoint, 
                                                   attitude_feedback=attitude_feedback,
                                                   rate_feedback=rate_feedback, 
                                                   dt=dt)
        
        return self.thrust_setpoint, self.torque_setpoint  
    
    def update_position(self, 
                        position_setpoint=np.array([0.0, 0.0, 0.0]), 
                        position_feedback=np.array([0.0, 0.0, 0.0]), 
                        velocity_feedback=np.array([0.0, 0.0, 0.0]), 
                        attitude_feedback=np.array([0.0, 0.0, 0.0]),
                        rate_feedback=np.array([0.0, 0.0, 0.0]),
                        dt=0.01):        
        # Position control
        self.position_setpoint = position_setpoint
        self.velocity_setpoint = np.array([pid.update(error=error, dt=dt)
                                            for error, pid in zip(position_setpoint - position_feedback, 
                                                                  self.position_pid)])
        
        # Limit velocity setpoint
        self.velocity_setpoint = np.clip(self.velocity_setpoint, -3.0, 3.0)
        
        # Velocity + Attitude + Rate control
        self.thrust_setpoint, self.torque_setpoint = self.update_velocity(velocity_setpoint=self.velocity_setpoint, 
                                                    velocity_feedback=velocity_feedback,
                                                    attitude_feedback=attitude_feedback,
                                                    rate_feedback=rate_feedback, 
                                                    dt=dt)
                
        return self.thrust_setpoint, self.torque_setpoint 

In [8]:
class Drone:
    def __init__(self):
        self.actuators  = Actuators(geometry='bicopter') # also possible: 'hexacopter H'
        self.dynamics   = Dynamics()
        self.controller = Cascade_controller()
        self.allocation = Control_allocation(self.actuators)
        
    def update(self, 
               setpoint=np.array([0.0, 0.0, 0.0]),
               setpoint_type='attitude',
               dt=0.01):
        # Get sensor data (perfect measurement, plus added noise)
        position = self.dynamics.position + 0.0 * np.random.randn(3)
        velocity = self.dynamics.velocity + 0.0 * np.random.randn(3)
        attitude = self.dynamics.attitude + 0.0 * np.random.randn(3)
        rate     = self.dynamics.rate     + 0.0 * np.random.randn(3)
        
        # Update control
        if setpoint_type == 'position':
            # Torque and thrust are computed from Position, Velocity, Attitude and Rate PIDs
            thrust, torque = self.controller.update_position(position_setpoint=setpoint,
                                                             position_feedback=position,
                                                             velocity_feedback=velocity,
                                                             attitude_feedback=attitude,
                                                             rate_feedback=rate,
                                                             dt=dt)
        elif setpoint_type == 'velocity':
            # Torque and thrust are computed from Velocity, Attitude and Rate PIDs
            thrust, torque = self.controller.update_velocity(velocity_setpoint=setpoint,
                                                             velocity_feedback=velocity,
                                                             attitude_feedback=attitude,
                                                             rate_feedback=rate,
                                                             dt=dt)
        elif setpoint_type == 'attitude':
            # Torque is computed from Attitude and Rate PIDs
            torque = self.controller.update_attitude(attitude_setpoint=setpoint,
                                                     attitude_feedback=attitude,
                                                     rate_feedback=rate,
                                                     dt=dt)
            # Constant thrust setpoint
            thrust = np.array([0.0, 0.0, - self.dynamics.mass * 9.81])
            
        elif setpoint_type == 'rate':
            # Torque is computed from Rate PID
            torque = self.controller.update_rate(rate_setpoint=setpoint,
                                                 rate_feedback=rate,
                                                 dt=dt)
            # Constant thrust setpoint
            thrust = np.array([0.0, 0.0, - self.dynamics.mass * 9.81])
        
        else:
            print('Error: {} is not a valid setpoint type!'.format(setpoint_type))
            
        # Control allocation
        actuator_setpoint = self.allocation.compute_actuator_setpoint(thrust, torque)
        
        # Simulate actuators
        thrust, torque = self.actuators.compute_thrust_and_torque(actuator_setpoint)
        
        # Update dynamic model
        self.dynamics.update(thrust=thrust,
                             torque=torque,
                             dt=dt)

In [22]:
############
# Init drone
############
drone = Drone()

# Print geometry of drone
print('Actuator Effectiveness:\n',
      drone.actuators.effectiveness)
print('Control Allocation:\n', 
      drone.allocation.B.round(2))

# Prepare logging
n_steps = 500
dt = 0.03
time = np.linspace(0.0, dt * (n_steps - 1), n_steps)

log = {
    'time': time,
    'position': np.zeros([n_steps, 3]),
    'velocity': np.zeros([n_steps, 3]),
    'attitude': np.zeros([n_steps, 3]),
    'rate': np.zeros([n_steps, 3]),
    'position_setpoint': np.zeros([n_steps, 3]),
    'velocity_setpoint': np.zeros([n_steps, 3]),
    'attitude_setpoint': np.zeros([n_steps, 3]),
    'rate_setpoint': np.zeros([n_steps, 3]),
    'torque_setpoint': np.zeros([n_steps, 3]),
    'thrust_setpoint': np.zeros([n_steps, 3]),
    'actuator_setpoint': np.zeros([n_steps, drone.allocation.B.shape[0]])
}

def update_log(i, log, drone):
    # Log drone state
    log['position'][i] = drone.dynamics.position
    log['velocity'][i] = drone.dynamics.velocity
    log['attitude'][i] = drone.dynamics.attitude
    log['rate'][i]     = drone.dynamics.rate
    
    # Log controller setpoints 
    log['position_setpoint'][i] = drone.controller.position_setpoint
    log['velocity_setpoint'][i] = drone.controller.velocity_setpoint
    log['attitude_setpoint'][i] = drone.controller.attitude_setpoint
    log['rate_setpoint'][i]     = drone.controller.rate_setpoint
    log['torque_setpoint'][i]   = drone.allocation.torque_setpoint
    log['thrust_setpoint'][i]   = drone.allocation.thrust_setpoint
    log['actuator_setpoint'][i] = drone.allocation.actuator_setpoint
 

Actuator Effectiveness:
 [[ 1.5 -1.5]
 [ 0.   0. ]
 [ 0.   0. ]
 [-3.  -3. ]]
Control Allocation:
 [[ 0.33  0.    0.   -0.17]
 [-0.33  0.    0.   -0.17]]


In [23]:
################
# Run simulation
################

# Type of setpoint
# setpoint_type = 'rate'
# setpoint_type = 'attitude'
# setpoint_type = 'velocity'
setpoint_type = 'position'

# Setpoint
setpoint = np.zeros([n_steps, 3])
if setpoint_type in ['attitude', 'rate']:
    setpoint[(1.0 < time) & (time < 4.0)] = np.array([0.25, 0.0, 0.0])
    setpoint[(4.0 < time) & (time < 8.0)] = np.array([-0.25, 0.0, 0.0])
elif setpoint_type in ['velocity']:
    setpoint[(1.0 < time) & (time < 4.0)] = np.array([0.0, 0.5, 0.0])
    setpoint[(4.0 < time) & (time < 8.0)] = np.array([0.0, -0.5, 0.0])
elif setpoint_type in ['position']:
    setpoint[(1.0 < time) & (time < 8.0)] = np.array([0.0, 0.5,  0.0])
    
for i in range(n_steps):
    # Update simulation
    drone.update(dt=dt, 
                 setpoint=setpoint[i],
                 setpoint_type=setpoint_type)
    
    # Log
    update_log(i, log, drone)

In [24]:
######
# Plot
######

# Create figure
fig = plt.figure(figsize=[15, 10])
fig.tight_layout()

# Subplots
gs = gridspec.GridSpec(3, 3)
axes = {
    'position': plt.subplot(gs[1, 0]),
    'velocity': plt.subplot(gs[2, 0]),
    'attitude': plt.subplot(gs[1, 1]),
    'rate': plt.subplot(gs[2, 1]),
    'thrust_torque': plt.subplot(gs[1, 2]),
    'actuator': plt.subplot(gs[2, 2]),
    'drone': plt.subplot(gs[0, :]),
}
 
time  = log['time']

# Plot drone states and setpoints
for k, name in enumerate(['position', 'velocity', 'attitude', 'rate']):
    ax = axes[name]
    state    = log[name]
    setpoint = log['{}_setpoint'.format(name)]
    
    # check if setpoint should be displayed
    do_display_setpoint = (setpoint.sum() != 0.0)
        
    for i in range(3):
        color = 'C{}'.format(i)
        label = ['X', 'Y', 'Z'][i]
        
        if (label == 'X' and (name == 'attitude' or name == 'rate')) or \
           (label in ['Y', 'Z'] and (name == 'position' or name == 'velocity')):
            ax.plot(time, state[:,i],
                       label='{} {}'.format(label,name), color=color)
            if do_display_setpoint:
                ax.plot(time, setpoint[:,i],
                           label='{} setpoint'.format(label),
                           color=color, linestyle='--')
    ax.legend(loc='upper right')
    ax.set_xlim(log['time'].min(), log['time'].max())

# Display actuator setpoints
ax = axes['actuator']
for i in range(log['actuator_setpoint'].shape[1]):
    ax.plot(log['time'], log['actuator_setpoint'][:, i], label='actuator {}'.format(i))
ax.set_xlim(log['time'].min(), log['time'].max())
ax.set_ylim(0, 1)
ax.legend(loc='upper right')

# Display thrust and torque setpoint
ax = axes['thrust_torque']
ax.plot(log['time'], log['torque_setpoint'][:, 0], label='torque')
ax.plot(log['time'], log['thrust_setpoint'][:, 2], label='thrust')
ax.set_xlim(log['time'].min(), log['time'].max())
ax.set_ylim(-5, 5)
ax.legend(loc='upper right')

def drone_marker(angle):
    radius = 0.2
    codes = [Path.MOVETO] + [Path.LINETO]*3 + [Path.CLOSEPOLY]
    vertices = [(-radius, -1.0*radius), (-radius, 0.0*radius), (radius, 0.0*radius), (radius, -1.0*radius), (0, 0)]
    # Rotors
    for rotor in drone.actuators.rotors:
        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices += [(rotor.position[1], rotor.position[2] - 0.2 * radius), 
                     (rotor.position[1] - radius, rotor.position[2]),
                     (rotor.position[1] + radius, rotor.position[2]), 
                     (0, 0)]
    vertices = np.array(vertices)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle), np.cos(angle)]])
    vertices = R.dot(vertices.T).T
    drone_path = Path(vertices, codes)
    return drone_path

# Display drone
ax = axes['drone']
# ax.set_xticks([])
# ax.set_yticks([])
# ax.axis('equal')
(drone_drawing,) = ax.plot([log['position'][0, 1]], 
                           [log['position'][0, 2]], 
                           marker=drone_marker(log['attitude'][0, 0]), markersize=50)
ax.set_xlim(log['position'][:, 1].min() - 2, 
            log['position'][:, 1].max() + 2)
ax.set_ylim(log['position'][:, 2].min() - 2, 
            log['position'][:, 2].max() + 2)

# Elements to be animated
(y_position_dot,) = axes['position'].plot(log['time'][0], log['position'][0, 1], marker='o', color='C1')
(z_position_dot,) = axes['position'].plot(log['time'][0], log['position'][0, 2], marker='o', color='C2')
(y_velocity_dot,) = axes['velocity'].plot(log['time'][0], log['velocity'][0, 1], marker='o', color='C1')
(z_velocity_dot,) = axes['velocity'].plot(log['time'][0], log['velocity'][0, 2], marker='o', color='C2')
(x_attitude_dot,) = axes['attitude'].plot(log['time'][0], log['attitude'][0, 0], marker='o')
(x_rate_dot,)     = axes['rate'].plot(log['time'][0], log['rate'][0, 0], marker='o')
(x_torque_dot,)   = axes['thrust_torque'].plot(log['time'][0], log['torque_setpoint'][0, 0], marker='o', color='C0')
(x_thrust_dot,)   = axes['thrust_torque'].plot(log['time'][0], log['thrust_setpoint'][0, 2], marker='o', color='C1')
(x_actuator1_dot,) = axes['actuator'].plot(log['time'][0], log['actuator_setpoint'][0, 0], marker='o', color='C0')
(x_actuator2_dot,) = axes['actuator'].plot(log['time'][0], log['actuator_setpoint'][0, 1], marker='o', color='C1')

# Invert Y axis because Z is down
axes['position'].invert_yaxis()
axes['velocity'].invert_yaxis()
axes['drone'].invert_yaxis()

# animate function
frame_skip = 1
def animate(ii):
    i = frame_skip * ii
    
    # Update dots
    x_rate_dot.set_data(log['time'][i], log['rate'][i, 0])
    
    y_position_dot.set_data( log['time'][i], log['position'][i, 1])
    z_position_dot.set_data( log['time'][i], log['position'][i, 2])
    y_velocity_dot.set_data( log['time'][i], log['velocity'][i, 1])
    z_velocity_dot.set_data( log['time'][i], log['velocity'][i, 2])
    x_attitude_dot.set_data( log['time'][i], log['attitude'][i, 0])
    x_rate_dot.set_data(     log['time'][i], log['rate'][i, 0])
    x_torque_dot.set_data(   log['time'][i], log['torque_setpoint'][i, 0])
    x_thrust_dot.set_data(   log['time'][i], log['thrust_setpoint'][i, 2])
    x_actuator1_dot.set_data(log['time'][i], log['actuator_setpoint'][i, 0])
    x_actuator2_dot.set_data(log['time'][i], log['actuator_setpoint'][i, 1])

    # Update drone marker
    drone_drawing.set_data(log['position'][i, 1], log['position'][i, 2])
    drone_drawing.set_marker(drone_marker(log['attitude'][i, 0]))
    
    # Center drone view
#     axes['drone'].set_xlim(log['position'][i, 1] - 2, log['position'][i, 1] + 2)
#     axes['drone'].set_ylim(log['position'][i, 2] - 2, log['position'][i, 2] + 2)
    
    return (x_rate_dot, drone_drawing)

anim = animation.FuncAnimation(fig, animate, init_func=None,
                               frames=int(n_steps / frame_skip), 
                               interval=50 * frame_skip, blit=True)

plt.close(fig)

anim

In [25]:
anim.save('cascade_control_position_300dpi.mp4', dpi=300)
# anim.save('cascade_control_velocity_300dpi.mp4', dpi=300)
# anim.save('cascade_control_attitude_300dpi.mp4', dpi=300)
# anim.save('cascade_control_rate_300dpi.mp4', dpi=300)